<a href="https://colab.research.google.com/github/defyMiy/NMT-Project/blob/main/server.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

In [ ]:
!pip install simpletransformers
!pip install torch
!pip install pythainlp
!pip install python-crfsuite
!pip install sacremoses

In [ ]:
!rm -rf '/content/ngrok-stable-linux-amd64.tgz'
!rm -rf '/content/ngrok'

!pip install flask --quiet
!pip install flask-ngrok --quiet
!pip install -U flask-cors

# install ngrok linux version using the following command or you can get the
# latest version from its official website- https://dashboard.ngrok.com/get-started/setup
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.tgz

# extract the downloaded file using the following command 
!tar -xvf /content/ngrok-stable-linux-amd64.tgz

# paste your AuthToken here and execute this command
!./ngrok authtoken 2Q8ZedCqtQIP2l7e7KVREa2MMvc_2qRULQx4QRLYG97GMTyaZ

In [18]:
import logging
import torch
from simpletransformers.t5 import T5Model, T5Args
from simpletransformers.seq2seq import Seq2SeqModel, Seq2SeqArgs

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

model_args = T5Args()
model_args.max_length = 100
model_args.length_penalty = 1
model_args.num_beams = 10

model_args_s2s = Seq2SeqArgs()
model_args.max_length = 100
model_args.length_penalty = 1
model_args.num_beams = 10

device = torch.cuda.is_available()

In [19]:
import re
import pycrfsuite
from pythainlp import sent_tokenize

def split_sentence_TH(TH_predict):
  TH_predict = sent_tokenize(TH_predict)
  for i in range(len(TH_predict)):
    # remove space in back
    TH_predict[i] = TH_predict[i].rstrip()

    # debug for engine fail example ['750ml', '1500ml']
    # move to append previous sentences
    # and replace ''
    if re.search('[0-9]', TH_predict[i][0]):
      j = i
      while True:
        if TH_predict[j-1] != '':
          TH_predict[j-1] += ' ' + TH_predict[i]
          break
      j -= 1
      TH_predict[i] = ''

  # remove '' in list
  while '' in TH_predict:
    TH_predict.remove('')
  print(TH_predict)
  return TH_predict

def split_sentence_EN(EN_predict):
  EN_predict = EN_predict.split('. ')
  for i in range(len(EN_predict)):
    if EN_predict[i][-1] != '.':
      EN_predict[i] += '.'
  print(EN_predict)
  return EN_predict
  
def predict(machine, text):
  if machine == 'mt5':
    path = '/content/gdrive/MyDrive/model/mt5-4bs-4e-5lr'
    model = T5Model("mt5", path, args=model_args, use_cuda=device)
  elif machine == 'marian':
    path = '/content/gdrive/MyDrive/model/marian-add'
    model = Seq2SeqModel(encoder_decoder_type="marian", encoder_decoder_name=path, args=model_args_s2s, use_cuda=device)
  elif machine == 'mbart':
    path = '/content/gdrive/MyDrive/model/mbart-4bs'
    model = Seq2SeqModel(encoder_decoder_type="mbart", encoder_decoder_name=path, args=model_args_s2s, use_cuda=device)
  predictions = model.predict(text)
  print(predictions)
  return predictions

In [ ]:
import logging
# import Flask from flask module
from flask import Flask, request, jsonify
from flask_cors import CORS
# import run_with_ngrok from flask_ngrok to run the app using ngrok
from flask_ngrok import run_with_ngrok

app = Flask(__name__) #app name
logging.basicConfig(level=logging.INFO)
logging.getLogger('flask_cors').level = logging.DEBUG
CORS(app)
run_with_ngrok(app)
  
@app.route("/", methods=["GET", "POST"])
def index():
  if request.method == "POST":
    json_data = request.get_json()
    if not json_data:
      return jsonify({"error": "no data"})
    try:
      if json_data["task"] == "th-en":
        prediction = split_sentence_TH(json_data["text"])
      elif json_data["task"] == "en-th":
        prediction = split_sentence_EN(json_data["text"])
      prediction = predict(json_data["machine"], prediction)
      data_predicted = {"prediction": prediction}
      return jsonify(data_predicted)
    except Exception as e:
      return jsonify({"error": str(e)})
  return "OK"
  
if __name__ == "__main__":
  app.run()

DEBUG:flask_cors.extension:Configuring CORS with resources: {'/*': {'origins': ['.*'], 'methods': 'DELETE, GET, HEAD, OPTIONS, PATCH, POST, PUT', 'allow_headers': ['.*'], 'expose_headers': None, 'supports_credentials': False, 'max_age': None, 'send_wildcard': False, 'automatic_options': True, 'vary_header': True, 'resources': '/*', 'intercept_exceptions': True, 'always_send': True}}


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


 * Running on http://5030-35-192-146-164.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


DEBUG:flask_cors.extension:Request to '/' matches CORS resource '/*'. Using options: {'origins': ['.*'], 'methods': 'DELETE, GET, HEAD, OPTIONS, PATCH, POST, PUT', 'allow_headers': ['.*'], 'expose_headers': None, 'supports_credentials': False, 'max_age': None, 'send_wildcard': False, 'automatic_options': True, 'vary_header': True, 'resources': '/*', 'intercept_exceptions': True, 'always_send': True}
DEBUG:flask_cors.core:CORS request received with 'Origin' http://localhost:3000
DEBUG:flask_cors.core:The request's Origin header matches. Sending CORS headers.
DEBUG:flask_cors.core:Settings CORS headers: MultiDict([('Access-Control-Allow-Origin', 'http://localhost:3000'), ('Access-Control-Allow-Headers', 'content-type'), ('Access-Control-Allow-Methods', 'DELETE, GET, HEAD, OPTIONS, PATCH, POST, PUT'), ('Vary', 'Origin')])
INFO:werkzeug:127.0.0.1 - - [22/May/2023 17:40:31] "OPTIONS / HTTP/1.1" 200 -


...

Generating outputs:   0%|          | 0/1 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3732: FutureWarning: 
`prepare_seq2seq_batch` is deprecated and will be removed in version 5 of HuggingFace Transformers. Use the regular
`__call__` method to prepare your inputs and targets.

Here is a short example:

model_inputs = tokenizer(src_texts, text_target=tgt_texts, ...)

If you either need to use different keyword arguments for the source and target texts, you should do two calls like
this:

model_inputs = tokenizer(src_texts, ...)
labels = tokenizer(text_target=tgt_texts, ...)
model_inputs["labels"] = labels["input_ids"]

See the documentation of your specific tokenizer for more details on the specific arguments to the tokenizer of choice.
For a more complete example, see the implementation of `prepare_seq2seq_batch`.

  warnings.warn(formatted_warning, FutureWarning)


Decoding outputs:   0%|          | 0/6 [00:00<?, ?it/s]

DEBUG:flask_cors.extension:Request to '/' matches CORS resource '/*'. Using options: {'origins': ['.*'], 'methods': 'DELETE, GET, HEAD, OPTIONS, PATCH, POST, PUT', 'allow_headers': ['.*'], 'expose_headers': None, 'supports_credentials': False, 'max_age': None, 'send_wildcard': False, 'automatic_options': True, 'vary_header': True, 'resources': '/*', 'intercept_exceptions': True, 'always_send': True}
DEBUG:flask_cors.core:CORS request received with 'Origin' http://localhost:3000
DEBUG:flask_cors.core:The request's Origin header matches. Sending CORS headers.
DEBUG:flask_cors.core:Settings CORS headers: MultiDict([('Access-Control-Allow-Origin', 'http://localhost:3000'), ('Vary', 'Origin')])
INFO:werkzeug:127.0.0.1 - - [22/May/2023 17:40:48] "POST / HTTP/1.1" 200 -


['This research article presents the suitability of a logic differential leakage technique for multiple distribution inverters in order to prevent the leakage condition caused by increasing number of distribution inverters. In this paper, 5 distribution inverters are used.', 'The simulation results showed that if the distribution power unit increases until the power capacity of distribution power unit is equal to the same or close to the power capacity of all the load.', 'If the inverter of all 5 distribution power units uses a phase-plane technique, it will not detect the leakage.', 'By applying all analytical techniques, it can be detected within 0.24 second.', 'By applying all hydraulic techniques, the detector can be detected within 0.282 second.', 'And if combined the pasteurized technique, a static technique and a hybrid technique can detect within 0.315 seconds.']
